In [3]:
import sys
sys.path.insert(0, "../../")


In [1]:
import os 
import frontmatter
from dataclasses import dataclass, field

def read_frontmater(file_path):
    # Create temp file

    with open(test_file_path) as f:
        post = frontmatter.load(f)

    fm = {key: post.get(key) for key in post.keys()}
    fm.update({'urls': []})

    for kb in fm.get('linked_kbs'):
        # url = re.search("(?P<url>https?://[^\s]+)", kb).group("url")
        t = kb.split('](')
        t = t[0][1:]
        fm['urls'].append(t)

    del fm['linked_kbs']

    return fm


test_file_path = "kb_md/posts/10000ft_connector/index.qmd"

class PathExists(Exception):
    def __init__(self):
        super().__init__("path does not exist")


if not os.path.exists(test_file_path):
    raise  PathExists(Exception)
    
test_fm = read_frontmater(test_file_path)
test_fm.keys()

dict_keys(['title', 'url', 'article_id', 'views', 'created_date', 'last updated', 'urls'])

In [4]:
import datacrew.crawler.crawler as dcc
driver = dcc.driversetup(is_headless=False)


In [5]:
from selenium.webdriver.common.by import By

TOPIC_URL = "https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC/connecting-data-to-domo?language=en_US"
BASE_TOPIC_URL = "https://domo-support.domo.com/s/topic/"

test_page_source = dcc.pagesource(url=TOPIC_URL, driver=driver,
                                  element_type=By.CLASS_NAME, element_id="section-list-item"
                                  )


💤 loading https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC/connecting-data-to-domo?language=en_US 💤
Page https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC/connecting-data-to-domo?language=en_US is loaded within 10 seconds.


In [14]:
import datacrew.crawler.article as dca
import selenium.webdriver
from bs4 import BeautifulSoup


@dataclass(init=False)
class Article_Category(dca.Article):
    url: str
    base_url: str
    driver: selenium.webdriver

    is_success: bool = False
    article: dca.Article = field(default=None, repr=False)

    category_soup: BeautifulSoup = field(default=None, repr=False)
    category_url_ls: list[str] = field(default=None)

    title: str = None
    md_str: str = field(default=None, repr=False)
    category_id: str = None

    def __init__(self, url, base_url, driver):
        self.url = url
        self.base_url = base_url
        self.driver = driver

        soup = dcc.pagesource(driver=self.driver, url=self.url,
                              element_type=By.CLASS_NAME, element_id="section-list-item")

        if not soup:
            raise dcc.ArticleKB_GetSoupError(url=self.url)

        super().__init__(base_url=base_url, soup=soup)

        self.article = dca.Article(soup=soup, base_url=self.base_url)
        self.kb_url_ls = self.article.linked_url_ls

        try:
            self.process_kb_soup(soup)
            self.is_success = True

        except dca.ArticleKB_ProcessSoupError as e:
            print(e)


def process_kb_soup(self, soup: BeautifulSoup):
    parent_term = "page-header"
    parent_description_term = "page-header-description"

    table_term = "section-list"
    table_item_term = "section-list-item"

    table = soup.find_all(class_=[table_term])

    if not table or table == []:
        raise dca.ArticleKB_ProcessSoupError(
            url=self.url, search_term=search_term)

        tarticle = []
        for row in table:
            # print("❤️")

            cells = row.find(class_="slds-form-element__label")

            if list(cells.strings):
                content = row.find(class_="slds-form-element__control")
                tarticle.append((list(cells.strings)[0], content))

        kb_soup = dict(tarticle)
        self.kb_soup = kb_soup

        self.title = self.article.md_soup(kb_soup.get("Title"))

        self.md_str = self.article.md_soup(kb_soup.get("Article Body"))
        self.article_id = self.article.md_soup(kb_soup.get("Article Number"))
        self.views = self.article.md_soup(
            kb_soup.get("Article Total View Count"))
        self.created = parser.parse(
            self.article.md_soup(kb_soup.get("Article Created Date"))
        )

        self.last_updated = parser.parse(
            self.article.md_soup(kb_soup.get("First Published Date"))
        )

        self.get_images(
            test_base_url='https://domo-support.domo.com//servlet/rtaImage')

        return self.kb_soup


In [15]:
test_article = Article_Category(url=TOPIC_URL,
                 driver=driver,
                 base_url=BASE_TOPIC_URL)

💤 loading https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC/connecting-data-to-domo?language=en_US 💤
Page https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC/connecting-data-to-domo?language=en_US is loaded within 10 seconds.


AttributeError: 'Article_Category' object has no attribute 'process_kb_soup'

In [ ]:
import os

kb_path = "nbs/projects/kb_md/posts"
kb_path = "kb_md/posts/"

if not os.path.exists(kb_path):
    raise Exception('kb_path does not exist')

for path, subdirs, files in os.walk(kb_path):
    for name in files:
        if name != 'index.qmd':
            continue
        # print(path, name)
        file_path = os.path.join(path, name)
